---
layout: post
title: Blog Post 3
---

In this Blog Post, I’ll use webscraping to answer the following question: **What movie or TV shows share actors with your favorite movie or show?**

The idea of this question is that, if TV show Y has many of the same actors as TV show X, and you like X, you might also enjoy Y. This post has two parts. In the first, larger part, I’ll write a webscraper for finding shared actors on IMDB. In the second, smaller part, I’ll use the results from my scraper to make recommendations.

## 1. Setup

### 1.1. Locate the Starting IMDB Page

I will pick my favorite TV show, *The handmaid's tale*, and locate its IMDB page. Its IMDB page is at:

```
https://www.imdb.com/title/tt5834204/
```

Save this URL for a moment.

### 1.2 Initialize Your Project

#### 1. Create a new GitHub repository, and sync it with GitHub Desktop. This repository will house your scraper. You should commit and push each time you make significant changes to your code.

My repository can be found [here](https://github.com/JadenWSR/IMDB_scraper).

```
https://github.com/JadenWSR/IMDB_scraper
```

#### 2. Open a Anaconda Prompt in the location of your repository on your laptop, and type:

```
conda activate PIC16B
scrapy startproject IMDB_scraper
cd IMDB_scraper
```

<p align="center">
    This will create quite a lot of files. We will write the scraper by modifying files in this folder.
</p>

## 2. Write Your Scraper

To get started, we first need to create a file inside the `spiders` directory called `imdb_spider.py`. We can do this by adding the following lines to the file:

```python
# to run 
# scrapy crawl imdb_spider -o movies.csv

import scrapy

class ImdbSpider(scrapy.Spider):
    name = 'imdb_spider'
    
    start_urls = ['https://www.imdb.com/title/tt5834204/']
```

Here, `start_urls` is the URL corresponding to my favorite TV show, *The handmaid's tale*.

We have now created a class called `ImdbSpider`. Next, we are going to implement three parsing methods for the `ImdbSpider` class.

The first method is `parse(self, response)`. This method does not return any data. It will simply navigate you from the movie's main page to Cast&Crew Page, then call `parse_full_credits(self, response)` method to do the following operations. The Cast&Crew page would be the `start_urls` followed by `fullcredits/?ref_=tt_ql_cl`, and we will use `response.urljoin()` to merge them together. Here is how I implemented this method:

In [ ]:
def parse(self, response):
    """
    Assumptions: Sart on a movie page, and then navigate to the Cast & Crew page.
    Effects: Once there, the parse_full_credits(self,response) should be called, by specifying
             this method in the callback argument to a yielded scrapy.Request.
      The parse() method does not return any data. 
    """
    # navigate to the Cast & Crew page
    page = response.urljoin("fullcredits/?ref_=tt_ql_cl")
    # call parse_full_credits method
    yield scrapy.Request(url=page, callback=self.parse_full_credits)

To make this method work, we will need to write another method called `parse_full_credits(self, response)`. Similarly, this method also does not return any data. It will navigate from the Cast&Crew page to each actor's personal page, and call `parse_actor_page(self, response)` to do the next step. Here is how I implemented this method:

In [ ]:
def parse_full_credits(self, response):
    """
    Assumption: start on the Cast & Crew page.
    Effects: yield a scrapy.Request for the page of each actor listed on the page. Crew members are not included.
             The yielded request should specify the method parse_actor_page(self, response) should be called when
             the actor’s page is reached.
    The parse_full_credits() method does not return any data. 
    """
    # Redirect to each actor's page and call parse_actor_page method
    for url in [a.attrib["href"] for a in response.css("td.primary_photo a")]: # a list of relative paths, one for each actor
        yield scrapy.Request(url =  response.urljoin(url), callback = self.parse_actor_page)

To make the following operations work, we need the third method called `parse_actor_page(self, response)`. This is the method that will actually get data and return dictionaries containing the information of each actor and the name of their movie or tv shows. To get the text instead of a bunch of html code from the css selector, we will use `.get()` and `getall()` to extract the information we wanted. Here is how I implemented this method:

In [ ]:
def parse_actor_page(self, response):
     """
     Assumption: Start on the page of an actor.
     Effects: It should yield a dictionary with two key-value pairs, of the form {"actor" : actor_name, "movie_or_TV_name" : movie_or_TV_name}.
     The method should yield one such dictionary for each of the movies or TV shows on which that actor has worked.
    """
    # Get the actor name
    actor = response.css(".header .itemprop::text").get()
    # get the list of movie or TV name
    movie_or_TV_name = response.css("b a::text").getall()
    # yield one dictionary for each of the movies or TV shows
    for name in movie_or_TV_name:
        yield {
            'actor': actor,
            'movie_or_TV_name': name
        }

Great! Now er have created our web scraper. To run this scraper, we need to save the `imdb_scraper.py` and type this line to our command window:

```
scrapy crawl imdb_spider -o results.csv
```

This will create a `.csv` file called `results.csv` with a column for actors and a column for movies or TV shows in our current working directory. We will make our recommendation based on this csv file.

## 3. Make Recommendations

### 3.1 Create the Sorted List

Before we start making suggestions, we need to do some standard imports and read in the data we just scraped from the IMDB website.

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt

In [2]:
IMDB = pd.read_csv("results.csv")
IMDB.head()

,actor,movie_or_TV_name
0,Genevieve Adam,Mrs. America
1,Genevieve Adam,The Handmaid's Tale
2,Genevieve Adam,Double Crossed
3,Genevieve Adam,She Is Swimming
4,Genevieve Adam,She Is Swimming


Here’s the list I obtained for *The handmaid's tale*. Of course, most shows will “share” the most actors with themselves.

In [3]:
# Count the number of shared actors
SortedList = IMDB.groupby("movie_or_TV_name").count().reset_index()
# Update column names
SortedList = SortedList.rename({"movie_or_TV_name":"movie", "actor":"number of shared actors"}, axis='columns')
# Sort by number of shared actors and update index
SortedList = SortedList.sort_values(["number of shared actors"],ascending=False).reset_index(drop=True)
SortedList.head(15)

,movie,number of shared actors
0,The Handmaid's Tale,522
1,Murdoch Mysteries,102
2,Designated Survivor,74
3,Air Crash Investigation,68
4,The Expanse,59
5,Saving Hope,59
6,Reign,57
7,Private Eyes,49
8,American Gods,45
9,Flashpoint,45


### 3.2 Visualization using networkx

We could create a nice visualization using networkx showing how the actors are shared in those movies. Since we have quite a lot of actors(522) in *The Handmaid's Tale*, I would just remove it for illustration purpose. I will only choose the movies with 60 or more shared actors since it's not possible to show every movie with actor links in one single plot. To make this plot, we would need to do some transformations to our IMDB data. In order to make modification without changing the original dataset, we make a copy of it.

In [4]:
results = IMDB.copy()

In [5]:
# Count the number of shared actors
results["count"] = results.groupby("movie_or_TV_name").transform(len)
# Filter out the movie with less than 60 shared actors
results = results[results["count"] > 60]
# Remove the handmaid's tale
results = results[np.invert(results["movie_or_TV_name"].str.contains("The Handmaid's Tale"))]
results.head()

,actor,movie_or_TV_name,count
29,Rohan Mead,Murdoch Mysteries,102
47,Ordena Stephens,Designated Survivor,74
106,Chris Gillett,Air Crash Investigation,68
434,Mckenna Grace,Designated Survivor,74
878,Danielle Height,Designated Survivor,74


With the filtered results, we should now be able to make the plot:

In [7]:
G = nx.from_pandas_edgelist(results, source = "actor", target = "movie_or_TV_name")
ranks = nx.pagerank(G)

In [ ]:
fig, ax = plt.subplots(1, figsize = (30, 12))
pos = nx.fruchterman_reingold_layout(G)
nx.draw(G, 
        pos, 
        node_size= 200*np.array([ranks[v] for v in G.nodes()]), 
        with_labels = True, 
        edge_color="lightgrey",
        font_size = 7,
        node_color = "lightblue",
        arrows = True)

To create this example, I first ran the code in a Jupyter Notebook, and added the line `plt.savefig("Blog-Post-3-Plot1.png")` to save the result. I then moved the file Blog-Post-3-Plot1.png to the images/ directory of my blog. Finally, I added the line:

```
![image-example.png]({{ site.baseurl }}/images/image-Blog-Post-3-Plot1.png) 
```
immediately beneath the code block.

Congratulations! By modifying the filter and the code above, you should be able to make linked actor plots for any movies at your choice.